In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Natural_Language_Processing')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Natural_Language_Processing


In [4]:
# set up the session info

from pyspark.sql import SparkSession

name = 'nlp'

spark = SparkSession.builder.appName(name).getOrCreate()


In [5]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
# rename the columns to more meanigful names

data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
# nlp tools
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, length
from pyspark.sql.types import IntegerType

In [8]:
data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [9]:
# check if langth of ham and spam differ

data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [10]:
from pyspark.ml.feature import StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [13]:
# make the transformation functions
# make the tokens

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

# remove stopwords
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol='stop_token')

# count
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')

# apply IDF
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

# convert ham/spam to integers
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [14]:
# get the assembler

from pyspark.ml.feature import VectorAssembler

In [15]:
# apply the functions

clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [16]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [17]:
# make a pipeline

from pyspark.ml import Pipeline

In [19]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [20]:
# apply the pipeline

cleaner = data_prep_pipe.fit(data)

In [23]:
clean_data = cleaner.transform(data)

In [24]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [25]:
clean_data = clean_data.select('label', 'features')
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [27]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [28]:
# fit the training model

spam_detector = nb.fit(training)

In [29]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [30]:
# run on test

test_results = spam_detector.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,5,20,...|[-805.15281254994...|[1.0,2.8585807635...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-874.77565255923...|[1.0,2.0673873034...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-692.95829661162...|[1.0,1.3185279391...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-615.62089964977...|[0.99996064389040...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-250.55545933351...|[0.68464346236640...|       0.0|
|  0.0|(13424,[0,1,498,5...|[-323.46928169124...|[0.99999999960567...|       0.0|
|  0.0|(13424,[0,1,874,1...|[-98.894936795809...|[0.99999995375415...|       0.0|
|  0.0|(13424,[0,2,3,5,3...|[-511.68494781184...|[1.0,1.1491433419...|       0.0|
|  0.0|(13424,[0,2,3,6,9...|[-3408.1833834434...|[1.0,1.4413755328...|       0.0|
|  0.0|(13424,[0

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [32]:
# use the accuracy

acc_eval = MulticlassClassificationEvaluator()

In [33]:
# accuracy

acc = acc_eval.evaluate(test_results)

In [34]:
print('Acc of NB Model')
print(acc)

Acc of NB Model
0.9290478626143888


In [35]:
labelAndPreds = test_results.select('label', 'prediction')

In [38]:
metrics = BinaryClassificationEvaluator()

In [39]:
evaluation = metrics.evaluate(test_results)

In [40]:
print("evaluation (area under ROC): %f" % evaluation)

evaluation (area under ROC): 0.177578


In [41]:
test_results.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

